In [1]:
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
x1=np.load('/content/drive/MyDrive/DataSets/Topo_Symm_Compounds_Normal_Nov23.npy')
x2=np.load('/content/drive/MyDrive/DataSets/Topo_Symm_Compounds_Swapped_Nov23.npy')
x3=np.load('/content/drive/MyDrive/DataSets/Topo_Vortex_Compounds_Normal_Nov23.npy')
x4=np.load('/content/drive/MyDrive/DataSets/Topo_Vortex_Compounds_Swapped_Nov23.npy')
y1=np.load('/content/drive/MyDrive/DataSets/Topo_Classifier_Output_Symm.npy')
y2=np.load('/content/drive/MyDrive/DataSets/Topo_Classifier_Output_Vortex.npy')
print(np.shape(x1))
print(np.shape(x2))
print(np.shape(x3))
print(np.shape(x4))
print(np.shape(y1))
print(np.shape(y2))

(187, 80, 512)
(187, 80, 512)
(294, 80, 512)
(294, 80, 512)
(187,)
(294,)


In [3]:
y=np.concatenate((y1,y2,y1,y2))
x=np.concatenate((x1,x3,x2, x4))
x=(x+1)/2
print(np.shape(x))
print(np.shape(y))

(962, 80, 512)
(962,)


In [4]:
def make_bin(yy):
  if yy==0:
    vec=1.0
  if yy==1:
    vec=0.0
  if yy==2:
    vec=0.0
  return vec
def vecty(yy):
  if yy==0:
    vec=[1,0,0]
  if yy==1:
    vec=[0,1,0]
  if yy==2:
    vec=[0,0,1]
  return vec



In [5]:
import random
xr=[]
yr=[]
lst=random.sample(range(int(962)), int(962))

for j in lst:
  yr.append(make_bin(y[j]))
  xr.append(x[j])

In [6]:
ns=962
print(ns)

x_train=xr[:int(.8*ns)]
x_test=xr[int(.8*ns):int(.9*ns)]
x_val=xr[int(.9*ns):]
y_train=yr[:int(.8*ns)]
y_test=yr[int(.8*ns):int(.9*ns)]
y_val=yr[int(.9*ns):]

962


In [7]:
x_train=np.array(x_train).reshape(int(.8*ns),160,256)
x_test=np.array(x_test).reshape(int(.1*ns),160,256)
x_val=np.array(x_val).reshape(97,160,256)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [8]:
run=True
while run==True:
  import keras
  from keras import layers
  import keras.backend as K
  from keras.layers import LeakyReLU
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=120, restore_best_weights=True, start_from_epoch=20)
  input_img = keras.Input(shape=( 160, 256, 1))
  x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img)
  x = LeakyReLU(alpha=0.2)(x)
  x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = layers.Conv2D(32, 3, strides = (2,2), padding='same')(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = layers.Flatten()(x)

  output= layers.Dense(1, activation='sigmoid')(x)

  crstldisc = keras.Model(input_img, output)
  crstldisc.compile(optimizer='adam', loss=K.binary_crossentropy, metrics=['accuracy'])
  history=crstldisc.fit(x_train, y_train,
                epochs=600,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, y_test), callbacks=[callback])
  if np.max(history.history['val_accuracy'])>0.88:
    run=False

Epoch 1/600
7/7 [==============================] - 3s 259ms/step - loss: 0.6960 - accuracy: 0.5254 - val_loss: 0.7228 - val_accuracy: 0.4167
Epoch 2/600
7/7 [==============================] - 2s 218ms/step - loss: 0.7164 - accuracy: 0.4811 - val_loss: 0.7092 - val_accuracy: 0.4167
Epoch 3/600
7/7 [==============================] - 1s 213ms/step - loss: 0.6983 - accuracy: 0.4811 - val_loss: 0.7028 - val_accuracy: 0.4167
Epoch 4/600
7/7 [==============================] - 2s 244ms/step - loss: 0.6968 - accuracy: 0.4811 - val_loss: 0.7069 - val_accuracy: 0.4167
Epoch 5/600
7/7 [==============================] - 2s 328ms/step - loss: 0.6989 - accuracy: 0.4811 - val_loss: 0.7145 - val_accuracy: 0.4167
Epoch 6/600
7/7 [==============================] - 2s 309ms/step - loss: 0.7031 - accuracy: 0.4811 - val_loss: 0.7202 - val_accuracy: 0.4167
Epoch 7/600
7/7 [==============================] - 2s 329ms/step - loss: 0.7048 - accuracy: 0.4811 - val_loss: 0.7079 - val_accuracy: 0.4167
Epoch 8/600
7

In [9]:
y=np.concatenate((y1,y2,y1,y2))
x=np.concatenate((x1,x3,x2, x4))
x=(x+1)/2
print(np.shape(x))
print(np.shape(y))

(962, 80, 512)
(962,)


In [10]:
import random
xr=[]
yr=[]
lst=random.sample(range(int(962)), int(962))

for j in lst:
  yr.append(vecty(y[j]))
  xr.append(x[j])

In [11]:
ns=962
print(ns)

x_train=xr[:int(.8*ns)]
x_test=xr[int(.8*ns):int(.9*ns)]
x_val=xr[int(.9*ns):]
y_train=yr[:int(.8*ns)]
y_test=yr[int(.8*ns):int(.9*ns)]
y_val=yr[int(.9*ns):]

962


In [12]:
x_train=np.array(x_train).reshape(int(.8*ns),160,256)
x_test=np.array(x_test).reshape(int(.1*ns),160,256)
x_val=np.array(x_val).reshape(97,160,256)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [13]:
import keras
from keras import layers
import keras.backend as K
from keras.layers import LeakyReLU
callback = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=120, restore_best_weights=True, start_from_epoch=60)

input_img = keras.Input(shape=( 160, 256, 1))

x = layers.Conv2D(4, 2, strides = (2,4),  padding='same')(input_img)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv2D(32, 2,strides = (2,2),  padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv2D(64, 3, strides = (2,2), padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)


x = layers.Flatten()(x)
output= layers.Dense(3, activation='softmax')(x)

crstldisc = keras.Model(input_img, output)
crstldisc.compile(optimizer='adam', loss=K.categorical_crossentropy, metrics=['CategoricalAccuracy','accuracy'])
crstldisc.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 256, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 80, 64, 4)         20        
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 80, 64, 4)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 40, 32, 32)        544       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 40, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 16, 64)        18496     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 20, 16, 64)        0   